In [2]:
import pretty_midi

# Load MIDI file into PrettyMIDI object
midi_data = pretty_midi.PrettyMIDI('adj_sim_outputs\midi\simulation.mid')

# Retrieve the list of beat times
beat_times = midi_data.get_beats()

print(beat_times)

[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17. ]


In [49]:
# play the midi file in adj_sim_outputs\midi\output.mid
import pygame
pygame.init()
pygame.mixer.music.load('adj_sim_outputs\midi\simulation.mid')
pygame.mixer.music.play()
pygame.time.wait(5000) # let the music play for 5 seconds
pygame.mixer.music.stop() # stop the music
pygame.quit()


In [31]:
# play the midi file in adj_sim_outputs\midi\output.mid
import pygame
pygame.init()
pygame.mixer.music.load('adj_sim_outputs\midi\interesting_1.mid')
pygame.mixer.music.play()
pygame.time.wait(5000) # let the music play for 5 seconds
pygame.mixer.music.stop() # stop the music
pygame.quit()


In [16]:
# load the midi file with mido
import mido
mid = mido.MidiFile('adj_sim_outputs\midi\simulation.mid')

# check if the midi is within standards
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0: 
MetaMessage('set_tempo', tempo=308223, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='D', time=0)
program_change channel=0 program=0 time=0
note_on channel=0 note=73 velocity=51 time=0
note_on channel=0 note=58 velocity=62 time=1
note_on channel=0 note=85 velocity=66 time=3
note_off channel=0 note=58 velocity=62 time=6
note_on channel=0 note=20 velocity=62 time=5
note_off channel=0 note=85 velocity=66 time=7
note_on channel=0 note=73 velocity=66 time=6
note_on channel=0 note=23 velocity=56 time=8
note_off channel=0 note=20 velocity=62 time=10
note_on channel=0 note=109 velocity=62 time=9
note_off channel=0 note=73 velocity=66 time=12
note_on channel=0 note=74 velocity=51 time=10
note_off channel=0 note=74 velocity=51 time=15
note_on channel=0 note=33 velocity=51 time=14
note_off channel=0 note=23 velocity=56 time=16
note_on channel=0 note=58 velocity=56 time=15
not

In [ ]:
import mido
import glob
import torch
from datasets import generate_piano_roll  # Assuming this function is in datasets.py

root_dir = 'data\\maestro-v3.0.0'
sequence_length = 100
beats_length = 50
device = 'cpu'

file_list = sorted(glob.glob('data\\maestro-v3.0.0\\**\\*.midi', recursive=True))

for idx, midi_name in enumerate(file_list):
    print(f"Loading and preprocessing data for index {idx}")
    piano_roll, durations, beats = generate_piano_roll(midi_name, sequence_length, beats_length)
    piano_roll = torch.from_numpy(piano_roll).float().to(device)
    durations = torch.from_numpy(durations).float().to(device)
    beats = torch.from_numpy(beats).float().to(device)
    torch.save((piano_roll, durations, beats), f'data\\tensors\\data_{idx}.pt')

In [6]:
import numpy as np

def generate_piano_roll(midi_input, sequence_length=300, beats_length=50):
    # Check if input is a file path or a mido.MidiFile object
    if isinstance(midi_input, str):
        midi = mido.MidiFile(midi_input)
        pretty_midi_obj = pretty_midi.PrettyMIDI(midi_input)
    elif isinstance(midi_input, mido.MidiFile):
        midi = midi_input
        pretty_midi_obj = pretty_midi.PrettyMIDI(midi.filename)
    else:
        raise ValueError("midi_input must be a file path or a mido.MidiFile object")
    # Initialize piano roll array and duration array
    piano_roll = np.zeros((128, sequence_length))
    durations = np.zeros((128, sequence_length))

    # record total time
    total_time = 0

    # Convert MIDI events to piano roll representation
    my_time = 0
    note_on_time = np.zeros(128)  # to keep track of when each note was turned on
    for msg in midi:
        my_time += msg.time
        time_step = int(round(my_time))  # convert time to nearest time step
        total_time = time_step
        if time_step >= sequence_length :
            break  # stop if the sequence length is exceeded
        if msg.type == 'note_on':
            # We use time_step as the x axis (time step) and note number as the y axis
            piano_roll[msg.note, time_step] = msg.velocity
            note_on_time[msg.note] = time_step
        elif msg.type == 'note_off':
            note_off_time = int(round(note_on_time[msg.note]))
            durations[msg.note, note_off_time:time_step] = time_step - note_off_time

    # Generate beats
    beats = pretty_midi_obj.get_beats()

    # Ensure beats is of length beats_length
    if len(beats) < beats_length:
        # If beats is too short, pad it with zeros
        beats = np.pad(beats, (0, beats_length - len(beats)))
    elif len(beats) > beats_length:
        # If beats is too long, truncate it
        beats = beats[:beats_length]

    del pretty_midi_obj
    del midi

    return piano_roll, durations, beats, total_time

In [7]:
import mido
import pickle
import glob
import torch
import numpy as np
import pretty_midi

sequence_length = 300
training_size = 30

beats_length = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_list = sorted(glob.glob('data\\maestro-v3.0.0\\**\\*.midi', recursive=True))

preprocessed_data = []


for idx, midi_name in enumerate(file_list):
    if idx % 50 == 0:
        print(f"Loading and preprocessing data for index {idx}")
    piano_roll, durations, beats, total_time = generate_piano_roll(midi_name, sequence_length, beats_length)

    beats_tensor = torch.from_numpy(beats).float()

    number_of_training_samples = int(np.floor(total_time/training_size))
    for i in range(number_of_training_samples):
        start = i * training_size
        end = start + training_size
        piano_roll_slice = piano_roll[:, start:start+sequence_length]
        durations_slice = durations[:, start:start+sequence_length]

        # Check if the slice is of the correct shape
        if piano_roll_slice.shape[1] == training_size and durations_slice.shape[1] == training_size and i != 0:
            piano_roll_tensor = torch.from_numpy(piano_roll_slice).float()
            durations_tensor = torch.from_numpy(durations_slice).float()
            preprocessed_data.append((piano_roll_tensor, durations_tensor, beats_tensor))


print("Saving preprocessed data")
with open('data\\preprocessed_data.pkl', 'wb') as f:
    pickle.dump(preprocessed_data, f)

Loading and preprocessing data for index 0
Loading and preprocessing data for index 50
Loading and preprocessing data for index 100
Loading and preprocessing data for index 150
Loading and preprocessing data for index 200
Loading and preprocessing data for index 250
Loading and preprocessing data for index 300
Loading and preprocessing data for index 350
Loading and preprocessing data for index 400
Loading and preprocessing data for index 450
Loading and preprocessing data for index 500
Loading and preprocessing data for index 550
Loading and preprocessing data for index 600
Loading and preprocessing data for index 650
Loading and preprocessing data for index 700
Loading and preprocessing data for index 750
Loading and preprocessing data for index 800
Loading and preprocessing data for index 850
Loading and preprocessing data for index 900
Loading and preprocessing data for index 950
Loading and preprocessing data for index 1000
Loading and preprocessing data for index 1050
Loading and

In [1]:
def generate_piano_roll(midi_input, sequence_length=None, beats_length=50, start=0, end=None):
    if sequence_length is None:
        sequence_length = end - start
    elif end is None:
        end = start + sequence_length
    # Check if input is a file path or a mido.MidiFile object
    if isinstance(midi_input, str):
        midi = mido.MidiFile(midi_input)
        pretty_midi_obj = pretty_midi.PrettyMIDI(midi_input)
    elif isinstance(midi_input, mido.MidiFile):
        midi = midi_input
        pretty_midi_obj = pretty_midi.PrettyMIDI(midi.filename)
    else:
        raise ValueError("midi_input must be a file path or a mido.MidiFile object")
    # Initialize piano roll array and duration array
    piano_roll = np.zeros((128, sequence_length))
    durations = np.zeros((128, sequence_length))

    # Convert MIDI events to piano roll representation
    my_time = 0
    note_on_time = np.zeros(128)  # to keep track of when each note was turned on
    for msg in midi:
        my_time += msg.time
        time_step = int(round(my_time))  # convert time to nearest time step
        if time_step >= sequence_length:
            break  # stop if the sequence length is exceeded
        if msg.type == 'note_on':
            # We use time_step as the x axis (time step) and note number as the y axis
            piano_roll[msg.note, time_step] = msg.velocity
            note_on_time[msg.note] = time_step
        elif msg.type == 'note_off':
            note_off_time = int(round(note_on_time[msg.note]))
            durations[msg.note, note_off_time:time_step] = time_step - note_off_time

    if end < len(piano_roll):
        piano_roll = piano_roll[:, start:end]
        durations = durations[:, start:end]
    else:
        # pad the piano roll and durations with zeros if the sequence length is not reached
        piano_roll = np.pad(piano_roll, ((0, 0), (0, sequence_length - piano_roll.shape[1])))
        durations = np.pad(durations, ((0, 0), (0, sequence_length - durations.shape[1])))

    # Generate beats
    beats = pretty_midi_obj.get_beats()

    # Ensure beats is of length beats_length
    if len(beats) < beats_length:
        # If beats is too short, pad it with zeros
        beats = np.pad(beats, (0, beats_length - len(beats)))
    elif len(beats) > beats_length:
        # If beats is too long, truncate it
        beats = beats[:beats_length]

    del pretty_midi_obj
    del midi

    return piano_roll, durations, beats


In [6]:
import matplotlib.pyplot as plt
import glob
import mido
import numpy as np
import pretty_midi

sequence_length = 100
beats_length = 50

file_list = sorted(glob.glob('data\\maestro-v3.0.0\\**\\*.midi', recursive=True))


# load random midi file from file_list
midi_name = file_list[0]
piano_roll, durations, beats = generate_piano_roll(midi_name, sequence_length, beats_length, start=30, end=60)


In [7]:
print(piano_roll.shape)
print(type(piano_roll))

(128, 30)
<class 'numpy.ndarray'>


In [8]:
print(piano_roll)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
# randome 10, 10 matrix
matrix = np.random.rand(10, 10)
size = 10
num_aug = 4
note_levels = np.ones(size - num_aug)
for i in range(size - num_aug):
    note_levels[i] = max(0,(int(matrix[size - num_aug + 2, i] * 126)) % 128)

print(note_levels)

[107.  72.  15.  29.  41.  14.]
